In [1]:
import warnings
warnings.filterwarnings(action="ignore")

import os
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pyrqa.settings import Settings
from pyrqa.time_series import TimeSeries
from pyrqa.computation import RPComputation
from pyrqa.metric import EuclideanMetric
from pyrqa.analysis_type import Classic
from pyrqa.neighbourhood import FixedRadius
from sklearn.metrics import mutual_info_score
from sklearn.preprocessing import StandardScaler
from scipy.stats import entropy
from scipy.linalg import svd
from tqdm import tqdm
from PIL import Image
from pyts.image import GramianAngularField

In [2]:
# Definir una función para calcular la información mutua
def mutual_information(x, y):
    x = x.ravel()  # Flatten the array to 1D
    y = y.ravel()  # Flatten the array to 1D
    c_xy = np.histogram2d(x, y, bins=32)[0]
    mi = mutual_info_score(None, None, contingency=c_xy)
    return mi

In [3]:
# Function to calculate false nearest neighbors
def false_nearest_neighbors(time_series, max_dim, tau, threshold=0.1):
    N = len(time_series)
    fnn_percentages = []
    
    for d in range(1, max_dim + 1):
        rtol = []
        atol = []
        for i in range(N - d * tau):
            if (i + d * tau) < N:
                distance = np.abs(time_series[i] - time_series[i + d * tau])
                if (i + (d + 1) * tau) < N:
                    rtol.append(distance / np.abs(time_series[i + (d + 1) * tau] - time_series[i]))
                    atol.append(np.abs(time_series[i + (d + 1) * tau] - time_series[i]))
        fnn_percentage = np.mean((np.array(rtol) > threshold) | (np.array(atol) > threshold))
        fnn_percentages.append(fnn_percentage)
        if fnn_percentage < threshold:
            break
    
    best_dim = np.argmin(fnn_percentages) + 1
    return best_dim, fnn_percentages

In [4]:
def calculate_entropy(matrix):
    """
    Calculate the Shannon entropy of a given matrix independent of binning.

    Parameters:
    matrix (numpy.ndarray): Input matrix for which to calculate entropy.

    Returns:
    float: Shannon entropy value.
    """
    # Flatten the matrix to a 1D array
    flat_matrix = matrix.flatten()

    # Calculate the frequency of each unique value in the flattened matrix
    value_counts = np.bincount(flat_matrix)

    # Calculate the probabilities
    probabilities = value_counts / np.sum(value_counts)

    # Filter out zero probabilities to avoid issues with log2(0)
    probabilities = probabilities[probabilities > 0]

    # Calculate the Shannon entropy
    shannon_entropy = entropy(probabilities, base=2)

    return shannon_entropy

In [5]:
def singular_value_spectrum(recurrence_matrix):
    u, s, vh = svd(recurrence_matrix)
    svs = np.log(s / np.sum(s))
    return svs, s

def principal_singular_value_proportion(svs):
    max_svs = np.max(svs)
    min_svs = np.min(svs)
    threshold = (max_svs + min_svs) / 2
    principal_svs = svs[svs >= threshold]
    psvp = len(principal_svs) / len(svs)
    return psvp

def calculate_psvp(recurrence_matrix):
    svs, s = singular_value_spectrum(recurrence_matrix)
    psvp = principal_singular_value_proportion(svs)
    return psvp, svs, s

In [6]:
def calculate_recurrence_plot(signal, optimal_delay, optimal_embedding, desired_rr):
    
    # Step 1: Create the TimeSeries object
    time_series = TimeSeries(signal,
                             embedding_dimension=optimal_embedding,
                             time_delay=optimal_delay)
    
    # Step 2: Define initial settings
    initial_radius = desired_rr  # Start with a small radius
    radius_step = 0.1  # Small step size for finer adjustment
    max_iterations = 100  # Max number of iterations to avoid infinite loop
    tolerance = 0.01  # Tolerance for recurrence rate difference

    # Function to calculate recurrence rate
    def compute_rr(radius):
        settings = Settings(time_series,
                            analysis_type=Classic,
                            similarity_measure=EuclideanMetric,
                            neighbourhood=FixedRadius(radius))
        computation = RPComputation.create(settings)
        rp = computation.run()
        recurrence_matrix = rp.recurrence_matrix_reverse
        rr = (recurrence_matrix.sum() - len(recurrence_matrix)) / (recurrence_matrix.size - len(recurrence_matrix))
        return rr, recurrence_matrix

    # Step 3: Iteratively adjust the radius to match the desired recurrence rate
    current_rr, recurrence_matrix = compute_rr(initial_radius)
    iteration = 0
    while abs(current_rr - desired_rr) > tolerance and iteration < max_iterations:
        if current_rr < desired_rr:
            initial_radius /= (1-radius_step)
        else:
            initial_radius *= (1-radius_step)
        current_rr, recurrence_matrix = compute_rr(initial_radius)
        #print(current_rr, initial_radius)
        iteration += 1

    # Return the recurrence matrix with the desired recurrence rate
    return recurrence_matrix

In [7]:
df_active_value = pd.read_csv('../data/redd_active_value_f1hz.csv')

In [8]:
attributes = [c for c in df_active_value.columns.values if not c in ['timestamp']]
labels = [c for c in df_active_value.columns.values if not c in ['timestamp', 'mains', 'amplitude_spectrum', 'phase_spectrum']]
#labels = ['Fridge01','Dish washer01','Microwave01','Washer dryer01','Washer dryer02']
predictors = ['mains']
index_name = 'timestamp'
training_start = '2011-04-16'
training_end = '2011-05-16'
test_start = '2011-05-17'
test_end = '2011-05-31'

window_sizes = [28, 56, 112, 224, 448, 896, 1792]
number_of_tests = 384
recurrence_rates = [0.05, 0.10, 0.15, 0.2, 0.25]

ts_image_type = ['rp','gasf','gadf']

results_file = '../data/results_rp_params_v5.csv'

In [9]:
# Ensure 'timestamp' column is in datetime format
df_active_value[index_name] = pd.to_datetime(df_active_value[index_name])
# Set the index as the timestamp
df_active_value.set_index(index_name, inplace=True)

In [10]:
# Partition the DataFrame into training and test sets
training_active_value_set = df_active_value.loc[training_start:training_end]
test_active_value_set = df_active_value.loc[test_start:test_end]


In [11]:
training_active_value_set.shape, test_active_value_set.shape

((2674113, 21), (1196394, 21))

In [12]:
scaler = StandardScaler()
scaler = scaler.fit(training_active_value_set)
training_active_value_set = scaler.transform(training_active_value_set)
training_active_value_set = pd.DataFrame(training_active_value_set)
training_active_value_set.columns = attributes

In [14]:
df_results = None
if os.path.exists(results_file):
    df_results = pd.read_csv(results_file)

for predictor in predictors:
    for window_size in window_sizes:
        rr_result = dict()
        rr_result['predictor'] = predictor
        print(rr_result)
        # calculate best time_delay based in predictor vs label
        time_step = int(training_active_value_set.shape[0] / number_of_tests)
        i, d = divmod(training_active_value_set.shape[0] - window_size + 1, time_step)
        row_factor = i + 1
        optimal_delay_list = list()
        for r in tqdm(range(int(row_factor))):
            if r * time_step >= window_size:
                signal = training_active_value_set.loc[r * time_step - window_size:r * time_step-1, [predictor]].values
                max_lag = int(window_size/2)  # Definir un máximo lag para considerar
                mi_values = [mutual_information(signal[:-lag], signal[lag:]) for lag in range(1, max_lag)]
                optimal_delay = np.argmax(mi_values) + 1
                optimal_delay_list.append(optimal_delay)
        optimal_delay_median = int(np.median(optimal_delay_list))

        rr_result['windows_size'] = window_size
        rr_result['optimal_delay'] = optimal_delay_median 
        print(rr_result)

        optimal_embedding_list = list()
        for r in tqdm(range(int(row_factor))):
            if r * time_step >= window_size:
                signal = training_active_value_set.loc[r * time_step - window_size:r * time_step-1, [predictor]].values.ravel()
                # Calculate the best embedding dimension using false nearest neighbors
                best_dim, fnn_percentages = false_nearest_neighbors(signal, 10, optimal_delay, 0.1) # type: ignore
                optimal_embedding_list.append(best_dim)
        optimal_embedding_median = int(np.median(optimal_embedding_list))

        rr_result['optimal_embedding'] = optimal_embedding_median 
        print(rr_result)

        # calculate best recurrence rate based predictor
        next_calc = True

        for image_type in ts_image_type:

            rr_result['image'] = image_type

            if image_type == 'rp':

                for rr in recurrence_rates:
                    if not df_results is None: # type: ignore
                        filter_result = (df_results['predictor'] == predictor) & (df_results['windows_size'] == window_size) & (rr_result['image'] == image_type) & (df_results['rr'] == rr) # type: ignore
                        if filter_result.sum() > 0:
                            next_calc = False
                        else:
                            next_calc = True
                    
                    if next_calc:
                        
                        rr_result['rr'] = rr                
                        coef1_list = list()
                        coef2_list = list()
                        coef3_list = list()
                        coef4_list = list()
                        coef5_list = list()
                        coef6_list = list()
                        
                        for r in tqdm(range(int(row_factor))):
                            if r * time_step >= window_size:
                                signal = training_active_value_set.loc[r * time_step - window_size:r * time_step-1, [predictor]].values
                                    
                                # Obtener la matriz de recurrencia
                                matrix = calculate_recurrence_plot(signal, optimal_delay_median, optimal_embedding_median, rr) # type: ignore                                                        

                                # Convert the matrix to an image
                                ts_image = Image.fromarray((matrix * 255).astype(np.uint8))

                                # Resize the image to 224x224
                                resized_image = ts_image.resize((224, 224), Image.LANCZOS)

                                # Convert the resized image back to a numpy array
                                resized_matrix = np.array(resized_image)

                                # Calcular la entropía y la PSVS
                                ent = calculate_entropy(resized_matrix) # type: ignore
                                PSVP,_,_ = calculate_psvp(resized_matrix) # type: ignore
                                
                                # Calcular la relación entre la entropía y la SVS
                                coef1 = ent / (PSVP + 0.0000001) 

                                # Calcular la relación entre la entropía y la SVS
                                coef2 = ent * PSVP

                                # Calcular la relación entre la entropía y la SVS
                                coef3 = (1 / (2**ent)) / (PSVP + 0.0000001)

                                # Calcular la relación entre la entropía y la SVS
                                coef4 = (1 / (2**ent)) * PSVP

                                # Add to results of coefs
                                coef1_list.append(coef1)
                                coef2_list.append(coef2)
                                coef3_list.append(coef3)
                                coef4_list.append(coef4)
                                coef5_list.append(ent)
                                coef6_list.append(PSVP)
                
                        rr_result['c_ent_div_psvp'] = np.mean(coef1_list)
                        rr_result['c_ent_mul_psvp'] = np.mean(coef2_list) 
                        rr_result['c_1_div_2_to_ent_div_psvp'] = np.mean(coef3_list)  
                        rr_result['c_1_div_2_to_ent_mul_psvp'] = np.mean(coef4_list)  
                        rr_result['c_ent'] = np.mean(coef5_list)
                        rr_result['c_psvp'] = np.mean(coef6_list)

                        print(rr_result) 
                        result_list = list()      
                        result_list.append(rr_result)

                        if os.path.exists(results_file):
                            result_list = pd.DataFrame(result_list)
                            result_list.to_csv(results_file, mode='a', header=False, index=False)
                        else:
                            result_list = pd.DataFrame(result_list)
                            result_list.to_csv(results_file, index=False)     

            else:                

                if not df_results is None: # type: ignore
                    filter_result = (df_results['predictor'] == predictor) & (df_results['windows_size'] == window_size) & (rr_result['image'] == image_type) # type: ignore
                    if filter_result.sum() > 0:
                        next_calc = False
                    else:
                        next_calc = True
                
                if next_calc:
                    
                    rr_result['rr'] = 0

                    coef1_list = list()
                    coef2_list = list()
                    coef3_list = list()
                    coef4_list = list()
                    coef5_list = list()
                    coef6_list = list()

                    for r in tqdm(range(int(row_factor))):
                        if r * time_step >= window_size:
                            signal = training_active_value_set.loc[r * time_step - window_size:r * time_step-1, [predictor]].values
                            
                            if image_type == 'gasf':                            
                                transformer = GramianAngularField(image_size=signal.shape[0], method='summation')
                                matrix = transformer.fit_transform(signal.reshape(1, -1))[0]
                            else:
                                transformer = GramianAngularField(image_size=signal.shape[0], method='difference')
                                matrix = transformer.fit_transform(signal.reshape(1, -1))[0]

                            # Convert the matrix to an image
                            ts_image = Image.fromarray((matrix * 255).astype(np.uint8))

                            # Resize the image to 224x224
                            resized_image = ts_image.resize((224, 224), Image.LANCZOS)

                            # Convert the resized image back to a numpy array
                            resized_matrix = np.array(resized_image)

                            # Calcular la entropía y la PSVS
                            ent = calculate_entropy(resized_matrix) # type: ignore
                            PSVP,_,_ = calculate_psvp(resized_matrix) # type: ignore
                            
                            # Calcular la relación entre la entropía y la SVS
                            coef1 = ent / (PSVP + 0.0000001) 

                            # Calcular la relación entre la entropía y la SVS
                            coef2 = ent * PSVP

                            # Calcular la relación entre la entropía y la SVS
                            coef3 = (1 / (2**ent)) / (PSVP + 0.0000001)

                            # Calcular la relación entre la entropía y la SVS
                            coef4 = (1 / (2**ent)) * PSVP

                            # Add to results of coefs
                            coef1_list.append(coef1)
                            coef2_list.append(coef2)
                            coef3_list.append(coef3)
                            coef4_list.append(coef4)
                            coef5_list.append(ent)
                            coef6_list.append(PSVP)

                    rr_result['c_ent_div_psvp'] = np.mean(coef1_list)
                    rr_result['c_ent_mul_psvp'] = np.mean(coef2_list) 
                    rr_result['c_1_div_2_to_ent_div_psvp'] = np.mean(coef3_list)  
                    rr_result['c_1_div_2_to_ent_mul_psvp'] = np.mean(coef4_list)  
                    rr_result['c_ent'] = np.mean(coef5_list)
                    rr_result['c_psvp'] = np.mean(coef6_list)   

                    print(rr_result) 
                    result_list = list()      
                    result_list.append(rr_result)

                    if os.path.exists(results_file):
                        result_list = pd.DataFrame(result_list)
                        result_list.to_csv(results_file, mode='a', header=False, index=False)
                    else:
                        result_list = pd.DataFrame(result_list)
                        result_list.to_csv(results_file, index=False)



                

                        

            


{'predictor': 'mains'}


100%|██████████| 385/385 [00:01<00:00, 301.53it/s]


{'predictor': 'mains', 'windows_size': 28, 'optimal_delay': 1}


100%|██████████| 385/385 [00:00<00:00, 1013.75it/s]


{'predictor': 'mains', 'windows_size': 28, 'optimal_delay': 1, 'optimal_embedding': 1}


...

{'predictor': 'mains', 'windows_size': 1792, 'optimal_delay': 1, 'optimal_embedding': 1, 'image': 'rp', 'rr': 0.25, 'c_ent_div_psvp': 2.404116304908813, 'c_ent_mul_psvp': 0.5015599092547696, 'c_1_div_2_to_ent_div_psvp': 146.912161001143, 'c_1_div_2_to_ent_mul_psvp': 0.026067542604804587, 'c_ent': 1.0395787343966587, 'c_psvp': 0.1691999254009698}


100%|██████████| 384/384 [00:56<00:00,  6.80it/s]


{'predictor': 'mains', 'windows_size': 1792, 'optimal_delay': 1, 'optimal_embedding': 1, 'image': 'gasf', 'rr': 0, 'c_ent_div_psvp': 27.76186029776464, 'c_ent_mul_psvp': 0.32080164312231435, 'c_1_div_2_to_ent_div_psvp': 146.85234155839146, 'c_1_div_2_to_ent_mul_psvp': 0.0034611549870946453, 'c_ent': 2.250342576528352, 'c_psvp': 0.050890525923162996}


100%|██████████| 384/384 [00:36<00:00, 10.54it/s]

{'predictor': 'mains', 'windows_size': 1792, 'optimal_delay': 1, 'optimal_embedding': 1, 'image': 'gadf', 'rr': 0, 'c_ent_div_psvp': 7.3057329063179095, 'c_ent_mul_psvp': 0.9904662555664798, 'c_1_div_2_to_ent_div_psvp': 6553524.810527114, 'c_1_div_2_to_ent_mul_psvp': 0.0008292602230581518, 'c_ent': 2.556533459791771, 'c_psvp': 0.13360220067139125}
